In [1]:
import re
import os
import time
import requests
import pandas as pd
from dotenv import load_dotenv; load_dotenv()
from supabase import create_client, Client

In [2]:
supabase: Client = create_client(
    os.environ.get("SUPABASE_URL"),
    os.environ.get("SUPABASE_KEY")
)

In [4]:
USER_ID = "c9e3aa85-b7f9-4ee7-b6ab-2673b46662aa"
user = supabase.auth.admin.get_user_by_id(USER_ID)

In [8]:
supabase.auth.set_session()

UserResponse(user=User(id='c9e3aa85-b7f9-4ee7-b6ab-2673b46662aa', app_metadata={'provider': 'github', 'providers': ['github']}, user_metadata={'avatar_url': 'https://avatars.githubusercontent.com/u/10243849?v=4', 'email': 'dennis@airgrid.io', 'email_verified': True, 'full_name': 'Dennis', 'iss': 'https://api.github.com', 'name': 'Dennis', 'phone_verified': False, 'preferred_username': 'ydennisy', 'provider_id': '10243849', 'sub': '10243849', 'user_name': 'ydennisy'}, aud='authenticated', confirmation_sent_at=None, recovery_sent_at=None, email_change_sent_at=None, new_email=None, invited_at=None, action_link=None, email='dennis@airgrid.io', phone='', created_at=datetime.datetime(2024, 6, 16, 14, 4, 2, 590913, tzinfo=TzInfo(UTC)), confirmed_at=datetime.datetime(2024, 6, 16, 14, 4, 2, 595941, tzinfo=TzInfo(UTC)), email_confirmed_at=datetime.datetime(2024, 6, 16, 14, 4, 2, 595941, tzinfo=TzInfo(UTC)), phone_confirmed_at=None, last_sign_in_at=datetime.datetime(2024, 6, 16, 14, 4, 26, 290729

In [ ]:
df = pd.read_csv("data/urls-20240616.csv")
df["text"] = df.subject + " " + df.body
texts = df.text.to_list()

In [ ]:
def extract_urls(text):
    url_pattern = re.compile(r'(?:(?:https?|ftp):\/\/)?[\w/\-?=%.]+\.[\w/\-?=%.]+')
    urls = re.findall(url_pattern, text)
    for i, url in enumerate(urls):
        if not url.startswith(("http://", "https://")):
            urls[i] = "http://" + url
    return urls

In [ ]:
results = []

for text in texts:
    try:
        urls = extract_urls(text)
        results.extend(urls)
    except Exception as ex:
        print(text, ex)

In [ ]:
final = pd.DataFrame({ "url": results })
final.to_csv("urls-processed-20240616.csv", index=False)

In [ ]:
token = os.getenv("TOKEN")
headers = {
    "Authorization": f"Bearer {token}"
}
for url in results:
    requests.post("http://localhost:8000/api/index", headers=headers, json={"urls": [url]})
    time.sleep(1)
